In [ ]:
#Installation of all the required lib here
!pip install oandapyV20
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib






     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 639.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for oandapyV20: filename=oandapyV20-0.7.2-py3-none-any.whl size=69779 sha256=e012c519270cf627987e12b00d8a9fcd9d1dfaa16220b557023800093e73954c
  Stored in directory: /root/.cache/pip/wheels/8e/69/ab/a6da84a85b9bf3b5a98ca15c6c52b7854f32b10c70fe0531a1
Successfully built oandapyV20
--2024-07-23 21:19:19--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-07-23 21:19:19--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.s

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import yfinance as yf
import talib

In [ ]:
class Indicator:
    def SMA_CROSS(self, data, short_period, long_period):
        data[f'SMA_{short_period}'] = talib.SMA(data['Close'].values, timeperiod=short_period)
        data[f'SMA_{long_period}'] = talib.SMA(data['Close'].values, timeperiod=long_period)
        return data



    def Baseline(self, data,period):
        data['baseline'] = data['Close'].rolling(window=period).mean()
        return data



    def SSL(self, data, period):
        data['smaHigh'] = data['High'].rolling(window=period).mean()
        data['smaLow'] = data['Low'].rolling(window=period).mean()
        data['sslDown'] = data.apply(lambda row: row['smaHigh'] if row['Close'] < row['smaLow'] else row['smaLow'], axis=1)
        data['sslUp'] = data.apply(lambda row: row['smaLow'] if row['Close'] < row['smaLow'] else row['smaHigh'], axis=1)
        data['Position'] = data.apply(lambda row: "buy" if row['smaLow'] == row['sslDown'] else "sell", axis=1)
        data['SIGNAL'] = data['Position'] != data['Position'].shift(1)
        data['SSL_BUY'] = (data['Position'] == 'buy') & (data['SIGNAL'])
        data['SSL_SELL'] = (data['Position'] == 'sell') & (data['SIGNAL'])
        return data


    def ATR(self, data, timeperiod=14):
        data['ATR'] = talib.ATR(data['High'].values, data['Low'].values, data['Close'].values, timeperiod=timeperiod)
        data['atrup'] = data['Close'] + data['ATR']
        data['atrdown'] = data['Close'] - data['ATR']

        # Calculate SL and TP based on the Position
        data['SL'] = data.apply(lambda row: row['Close'] - (row['ATR'] * 1.5) if row['Position'] == 'buy' else row['Close'] + (row['ATR'] * 1.5), axis=1)
        data['TP'] = data.apply(lambda row: row['Close'] + row['ATR'] if row['Position'] == 'buy' else row['Close'] - row['ATR'], axis=1)

        return data

    def WAE(self, df, sensitivity=150, fastLength=20, slowLength=40, channelLength=20, mult=2.0):
        df['Close'] = df['Close'].astype(float)
        df['macd'], _, _ = talib.MACD(df['Close'], fastperiod=fastLength, slowperiod=slowLength, signalperiod=9)

        # Manual Bollinger Bands calculation
        df['std'] = df['Close'].rolling(window=channelLength).std()
        df['bb_upper'] = df['baseline'] + (mult * df['std'])
        df['bb_lower'] = df['baseline'] - (mult * df['std'])

        df['t1'] = (df['macd'] - df['macd'].shift(1)) * sensitivity
        df['e1'] = df['bb_upper'] - df['bb_lower']

        df['trendUp'] = np.where(df['t1'] >= 0, df['t1'], 0)
        df['trendDown'] = np.where(df['t1'] < 0, -df['t1'], 0)
        def final_signal(row):
          long_conditon = row['Close'] > row['baseline'] and row['SSL_BUY'] and row['trendUp']>row['e1']
          short_conditon = row['Close'] < row['baseline'] and row['SSL_SELL'] and row['trendDown']>row['e1']

          if long_conditon:
              return "BUY"
          elif short_conditon:
              return "SELL"
          else:
              return None
        df['FINAL_SIGNAL'] = df.apply(final_signal,axis = 1)
        return df

In [ ]:

class backtest():
  def strategy(df):
      baseline_df = Indicator().Baseline(data=df, period=10)
      ssl_df = Indicator().SSL(data=baseline_df, period=10)
      atr_df = Indicator().ATR(data=ssl_df,timeperiod=14)
      wae_df=Indicator().WAE(df=atr_df)
      return wae_df


  def backtest_forex(df):
    entry_price = None
    trade_type = None
    pips_gained = []
    for index, row in df.iterrows():
      if row['FINAL_SIGNAL'] == 'SELL' or row['FINAL_SIGNAL'] == 'BUY':
        if entry_price is None:
          entry_price = row['Close']
          trade_type = row['FINAL_SIGNAL']
          print(entry_price,trade_type)
        elif entry_price is not None:

          exit_price = row['Close']
          pips = exit_price - entry_price if trade_type == "SELL" else entry_price - exit_price
          pips_gained.append(abs(pips))
          print(exit_price,entry_price,abs(pips))

          entry_price = row['Close']
          trade_type = row['FINAL_SIGNAL']
          print(entry_price,trade_type)

    return sum(pips_gained)



  def backtest_stock(df):
      collection = []           # Define inside the function to keep the state within the function scope
      collection_units = []     # Define inside the function to keep the state within the function scope
      gain_amount = []
      positive_count = 0        # Initialize the positive count
      negative_count = 0        # Initialize the negative count

      unit = 10
      for index, row in df.iterrows():
          if row['FINAL_SIGNAL'] == "BUY":
              investment = unit * row['Close']
              collection_units.append(unit)
              collection.append(investment)
          elif row['FINAL_SIGNAL'] == "SELL":
              if collection:
                  total_investment = sum(collection)
                  total_units = sum(collection_units)
                  total_selling_price = row['Close'] * total_units
                  gain = total_selling_price - total_investment
                  gain_amount.append(gain)
                  print(f"Sold Units: {total_units}")
                  print(f"Total investment: {total_investment}")
                  print(f"Total Selling Price: {total_selling_price}")
                  print(f"Gain from stock: {gain}")
                  collection = []  # Reset collection after selling
                  collection_units = []  # Reset units after selling
                  if gain > 0:
                      positive_count += 1
                  elif gain < 0:
                      negative_count += 1

      total_gain = sum(gain_amount)
      total_trades = positive_count + negative_count
      win_rate = (positive_count / total_trades * 100) if total_trades > 0 else 0

      print(f"Total Gain: {total_gain}")
      print(f"Positive Gains: {positive_count}")
      print(f"Negative Gains: {negative_count}")
      print(f"Win %: {win_rate}")

  # Example usage:
  # Assuming df is your DataFrame with the necessary columns
  # stock_bt(df)


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np  # Import numpy for np.where function

def create_figure_sig(sig):
    # Initialize figure with subplots
    fig_ssl = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

    # Candlestick chart
    fig_ssl.add_trace(go.Candlestick(
        x=sig['Time'],
        open=sig['Open'],
        high=sig['High'],
        low=sig['Low'],
        close=sig['Close'],
        name='Candlestick',
        line=dict(width=1),
        opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',
        decreasing_line_color='#FF3A4C'
    ), row=1, col=1)

    #TP and SL adder
    fig_ssl.add_trace(go.Scatter(

    ))

    # SSLUp, SSLDown, Baseline traces
    fig_ssl.add_trace(go.Scatter(
        x=sig['Time'],
        y=sig['sslUp'],
        name='sslUp',
        mode='lines',
        line=dict(color='green', width=2)
    ), row=1, col=1)

    fig_ssl.add_trace(go.Scatter(
        x=sig['Time'],
        y=sig['sslDown'],
        name='sslDown',
        mode='lines',
        line=dict(color='red', width=2)
    ), row=1, col=1)

    fig_ssl.add_trace(go.Scatter(
        x=sig['Time'],
        y=sig['baseline'],
        name='Baseline',
        mode='lines',
        line=dict(color='yellow', width=2)
    ), row=1, col=1)

    # Buy and Sell signals
    fig_ssl.add_trace(go.Scatter(
        x=sig[sig['FINAL_SIGNAL'] == "BUY"]['Time'],
        y=sig[sig['FINAL_SIGNAL'] == "BUY"]['Open'],
        name='BUY_ARROW',
        mode='markers',
        marker=dict(color='yellow', symbol='triangle-up', size=10)
    ), row=1, col=1)

    fig_ssl.add_trace(go.Scatter(
        x=sig[sig['FINAL_SIGNAL'] == "SELL"]['Time'],
        y=sig[sig['FINAL_SIGNAL'] == "SELL"]['Open'],
        name='SELL_ARROW',
        mode='markers',
        marker=dict(color='red', symbol='triangle-down', size=10)
    ), row=1, col=1)

    # WAE chart
    fig_ssl.add_trace(go.Bar(
        x=sig['Time'],
        y=sig['trendUp'],
        marker_color=np.where(sig['trendUp'] < sig['trendUp'].shift(1), 'lime', 'green'),
        name='UpTrend',
        width=0.04,
        marker_line_color='rgba(0,0,0,0)',
        marker_line_width=0,
    ), row=2, col=1)

    fig_ssl.add_trace(go.Bar(
        x=sig['Time'],
        y=sig['trendDown'],
        marker_color=np.where(sig['trendDown'] < sig['trendDown'].shift(1), 'orange', 'red'),
        name='DownTrend',
        width=0.04,
        marker_line_color='rgba(0,0,0,0)',
        marker_line_width=0,
    ), row=2, col=1)

    fig_ssl.add_trace(go.Scatter(
        x=sig['Time'],
        y=sig['e1'],
        mode='lines',
        name='ExplosionLine',
        line=dict(color='#A0522D', width=2)
    ), row=2, col=1)

    # Update layout for the entire figure
    fig_ssl.update_layout(
        title='Candlestick Chart with WAE',
        xaxis_title='Time',
        yaxis_title='Price',
        height=800,
        width=1200,
        margin=dict(l=50, r=50, b=50, t=100),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e",
        font=dict(size=10, color="#e1e1e1"),
        showlegend=True,
        legend=dict(x=0, y=1.1, orientation='h')
    )

    # Show gridlines
    fig_ssl.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,
        fixedrange=True,
        rangeslider=dict(visible=True)
    )

    fig_ssl.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )

    return fig_ssl






RG0H0IUW8F6K0XHT -API KEY


In [ ]:
!pip install alpha_vantage

In [ ]:
import requests
import json

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=RG0H0IUW8F6K0XHT'
response = requests.get(url)
data = response.json()

for i,k in data:
  for j,l in k:
    print(j)

ValueError: too many values to unpack (expected 2)

In [ ]:
df = yf.download(tickers="AAPL",period='max',interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [ ]:
str_df = strategy(df)

In [ ]:
str_df

,Open,High,Low,Close,Adj Close,Volume,baseline,smaHigh,smaLow,sslDown,...,TP,macd,std,bb_upper,bb_lower,t1,e1,trendUp,trendDown,FINAL_SIGNAL
Date,,,,,,,,,,,,,,,,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099058,469033600,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,None
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093890,175884800,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,None
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086998,105728000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,None
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089152,86441600,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,None
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091737,73449600,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-17,229.449997,231.460007,226.639999,228.880005,228.880005,57345900,229.358000,230.832001,226.605998,226.605998,...,234.027728,10.282387,9.574305,248.506610,210.209390,-23.282844,38.297220,0.0,23.282844,None
2024-07-18,230.279999,230.440002,222.270004,224.179993,224.179993,66034600,229.620999,231.721001,226.929999,231.721001,...,218.816392,9.882680,9.471859,248.564717,210.677281,-59.955973,37.887437,0.0,59.955973,SELL
2024-07-19,224.820007,226.800003,223.279999,224.309998,224.309998,49151500,229.417999,231.756001,227.092999,231.756001,...,219.078083,9.498809,9.079435,247.576869,211.259129,-57.580668,36.317740,0.0,57.580668,None


In [ ]:
str_df.columns

Index(['Time', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'baseline', 'smaHigh', 'smaLow', 'sslDown', 'sslUp', 'Position',
       'SIGNAL', 'SSL_BUY', 'SSL_SELL', 'ATR', 'atrup', 'atrdown', 'SL', 'TP',
       'macd', 'std', 'bb_upper', 'bb_lower', 't1', 'e1', 'trendUp',
       'trendDown', 'FINAL_SIGNAL'],
      dtype='object')

In [ ]:
str_df.rename(columns={"Date":"Time"},inplace=True)

In [ ]:
new_df = str_df.iloc[5000:5500]

In [ ]:
fig = create_figure_sig(new_df)
fig.show()

In [ ]:
buy_df = str_df[(str_df['FINAL_SIGNAL'] == 'BUY') | (str_df['FINAL_SIGNAL'] == 'SELL')]
ch = buy_df.iloc[10:30]

In [ ]:
str_

,Time,Open,High,Low,Close,Adj Close,Volume,baseline,smaHigh,smaLow,...,TP,macd,std,bb_upper,bb_lower,t1,e1,trendUp,trendDown,FINAL_SIGNAL
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099058,469033600,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,None
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093890,175884800,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,None
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086998,105728000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,None
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089152,86441600,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,None
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091737,73449600,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10988,2024-07-17,229.449997,231.460007,226.639999,228.880005,228.880005,57345900,229.358000,230.832001,226.605998,...,234.027728,10.282387,9.574305,248.506610,210.209390,-23.282844,38.297220,0.0,23.282844,None
10989,2024-07-18,230.279999,230.440002,222.270004,224.179993,224.179993,66034600,229.620999,231.721001,226.929999,...,218.816392,9.882680,9.471859,248.564717,210.677281,-59.955973,37.887437,0.0,59.955973,SELL
10990,2024-07-19,224.820007,226.800003,223.279999,224.309998,224.309998,49151500,229.417999,231.756001,227.092999,...,219.078083,9.498809,9.079435,247.576869,211.259129,-57.580668,36.317740,0.0,57.580668,None
10991,2024-07-22,227.009995,227.779999,223.089996,223.960007,223.960007,48201800,229.031999,231.749001,227.076999,...,218.766800,9.108050,8.407786,245.847572,212.216426,-58.613828,33.631146,0.0,58.613828,None


In [ ]:
def stock_bt(df):
    collection = []           # Define inside the function to keep the state within the function scope
    collection_units = []     # Define inside the function to keep the state within the function scope
    gain_amount = []
    positive_count = 0        # Initialize the positive count
    negative_count = 0        # Initialize the negative count

    unit = 10
    for index, row in df.iterrows():
        if row['FINAL_SIGNAL'] == "BUY":
            investment = unit * row['Close']
            collection_units.append(unit)
            collection.append(investment)
        elif row['FINAL_SIGNAL'] == "SELL":
            if collection:
                total_investment = sum(collection)
                total_units = sum(collection_units)
                total_selling_price = row['Close'] * total_units
                gain = total_selling_price - total_investment
                gain_amount.append(gain)
                print(f"Sold Units: {total_units}")
                print(f"Total investment: {total_investment}")
                print(f"Total Selling Price: {total_selling_price}")
                print(f"Gain from stock: {gain}")
                collection = []  # Reset collection after selling
                collection_units = []  # Reset units after selling
                if gain > 0:
                    positive_count += 1
                elif gain < 0:
                    negative_count += 1

    total_gain = sum(gain_amount)
    total_trades = positive_count + negative_count
    win_rate = (positive_count / total_trades * 100) if total_trades > 0 else 0

    print(f"Total Gain: {total_gain}")
    print(f"Positive Gains: {positive_count}")
    print(f"Negative Gains: {negative_count}")
    print(f"Win %: {win_rate}")

# Example usage:
# Assuming df is your DataFrame with the necessary columns
# stock_bt(df)


In [ ]:
stock_bt(str_df)

Sold Units: 80
Total investment: 8.415189906954765
Total Selling Price: 6.473199725151062
Gain from stock: -1.9419901818037033
Sold Units: 30
Total investment: 2.5613799691200256
Total Selling Price: 2.59488008916378
Gain from stock: 0.03350012004375458
Sold Units: 10
Total investment: 0.742190033197403
Total Selling Price: 0.7142899930477142
Gain from stock: -0.02790004014968872
Sold Units: 40
Total investment: 2.3660799860954285
Total Selling Price: 3.236599862575531
Gain from stock: 0.8705198764801025
Sold Units: 10
Total investment: 1.4676299691200256
Total Selling Price: 1.0825899988412857
Gain from stock: -0.38503997027873993
Sold Units: 30
Total investment: 2.845979928970337
Total Selling Price: 3.6495599895715714
Gain from stock: 0.8035800606012344
Sold Units: 20
Total investment: 2.4107200652360916
Total Selling Price: 2.4107199907302856
Gain from stock: -7.450580596923828e-08
Sold Units: 30
Total investment: 3.51004995405674
Total Selling Price: 3.7667401134967804
Gain from s

In [ ]:
import oandapyV20.endpoints.instruments as instruments

In [ ]:
instruments.Instru

TypeError: Can't instantiate abstract class Instruments with abstract method __init__

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
html = "https://finance.yahoo.com/trending-tickers/"
response = requests.get(html)
soup = BeautifulSoup(response.content,'html.parser')

In [ ]:
element = soup.find('div', class_=["Ovx(a)", "Ovx(h)--print", "Ovy(h)", "W(100%)"])
link = element.find('tbody').find('tr').find('a')
print(link.text)

TSLA


In [ ]:
tickers_list = []
element = soup.find('div', class_=["Ovx(a)", "Ovx(h)--print", "Ovy(h)", "W(100%)"])
tbodies = element.find_all('tbody')
for tbody in tbodies:
    rows = tbody.find_all('tr')
    for row in rows:
        link = row.find('a')
        # Do something with the link
        print(link.text)
        tickers_list.append(link.text)

TSLA
GOOG
V
GOOGL
UPS
ENPH
META
CMG
TXN
LMT
AZTR
RGC
ETHA
COF
HOLO
MA
GE
GM
SPOT
STX
TSLL
VSAT
DIS
FLYE
ADIL
BA
SLRX
ALLR
WM
CTNT


In [ ]:
liprint(ticker)

['TSLA', 'GOOG', 'V', 'GOOGL', 'UPS', 'ENPH', 'META', 'CMG', 'TXN', 'LMT', 'AZTR', 'RGC', 'ETHA', 'COF', 'HOLO', 'MA', 'GE', 'GM', 'SPOT', 'STX', 'TSLL', 'VSAT', 'DIS', 'FLYE', 'ADIL', 'BA', 'SLRX', 'ALLR', 'WM', 'CTNT']


In [ ]:
yf.download(tickers=ticker,period='max',interval='1d')

[*********************100%%**********************]  30 of 30 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ETHA']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


Price      Adj Close                                                         \
Ticker          ADIL   ALLR  AZTR          BA        CMG         COF   CTNT   
Date                                                                          
1962-01-02       NaN    NaN   NaN    0.190931        NaN         NaN    NaN   
1962-01-03       NaN    NaN   NaN    0.194749        NaN         NaN    NaN   
1962-01-04       NaN    NaN   NaN    0.192840        NaN         NaN    NaN   
1962-01-05       NaN    NaN   NaN    0.189022        NaN         NaN    NaN   
1962-01-08       NaN    NaN   NaN    0.189499        NaN         NaN    NaN   
...              ...    ...   ...         ...        ...         ...    ...   
2024-07-17      1.07  0.204  2.02  184.839996  55.290001  149.470001  0.472   
2024-07-18      1.06  0.187  1.85  180.229996  53.380001  148.720001  0.427   
2024-07-19      1.09  0.168  1.83  179.669998  53.540001  147.270004  0.435   
2024-07-22      1.06  0.164  1.62  178.899994  53.560001  145.110001  0.426   
2024-07-23      1.31  0.167  4.97  186.479996  52.549999  145.500000  0.471   

Price                                   ...      Volume                        \
Ticker            DIS        ENPH ETHA  ...        SLRX       SPOT        STX   
Date                                    ...                                     
1962-01-02   0.057941         NaN  NaN  ...         NaN        NaN        NaN   
1962-01-03   0.058719         NaN  NaN  ...         NaN        NaN        NaN   
1962-01-04   0.058719         NaN  NaN  ...         NaN        NaN        NaN   
1962-01-05   0.058914         NaN  NaN  ...         NaN        NaN        NaN   
1962-01-08   0.058719         NaN  NaN  ...         NaN        NaN        NaN   
...               ...         ...  ...  ...         ...        ...        ...   
2024-07-17  97.309998  111.150002  NaN  ...    168300.0  1982100.0  2825000.0   
2024-07-18  96.790001  107.949997  NaN  ...  21629800.0  1817300.0  2237600.0   
2024-07-19  95.739998  105.459999  NaN  ...    967900.0  1782400.0  1977200.0   
2024-07-22  94.129997  104.739998  NaN  ...    589400.0  3122100.0  2310900.0   
2024-07-23  90.940002  103.639999  NaN  ...  59986900.0  6640500.0  3779500.0   

Price                                                                  \
Ticker             TSLA        TSLL        TXN         UPS          V   
Date                                                                    
1962-01-02          NaN         NaN        NaN         NaN        NaN   
1962-01-03          NaN         NaN        NaN         NaN        NaN   
1962-01-04          NaN         NaN        NaN         NaN        NaN   
1962-01-05          NaN         NaN        NaN         NaN        NaN   
1962-01-08          NaN         NaN        NaN         NaN        NaN   
...                 ...         ...        ...         ...        ...   
2024-07-17  115584800.0  47106800.0  7792500.0   2863600.0  5324700.0   
2024-07-18  110869000.0  48474200.0  5497200.0   3412900.0  5550800.0   
2024-07-19   87403900.0  41125000.0  4868000.0   3073800.0  6839900.0   
2024-07-22  101225400.0  36129800.0  6063800.0   3629800.0  5313600.0   
2024-07-23  107565500.0  56178300.0  6057800.0  25332100.0  8063600.0   

Price                             
Ticker           VSAT         WM  
Date                              
1962-01-02        NaN        NaN  
1962-01-03        NaN        NaN  
1962-01-04        NaN        NaN  
1962-01-05        NaN        NaN  
1962-01-08        NaN        NaN  
...               ...        ...  
2024-07-17  2246700.0  1557700.0  
2024-07-18  2439600.0  1284000.0  
2024-07-19  1560000.0  1220900.0  
2024-07-22  1385900.0  1135000.0  
2024-07-23  2750600.0  1533600.0  

[15746 rows x 180 columns]

In [ ]:
data_dict = {}  # Dictionary to store DataFrames

for ticker in tickers_list:
    try:
        data_dict[ticker] = yf.download(tickers=ticker, period="max", interval="1mo")
        print(f"Data downloaded for {ticker}")
    except Exception as e:
        print(f"Failed to download data for {ticker}: {e}")



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Data downloaded for TSLA
Data downloaded for GOOG
Data downloaded for V


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Data downloaded for GOOGL
Data downloaded for UPS
Data downloaded for ENPH
Data downloaded for META


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...

Data downloaded for DIS
Data downloaded for FLYE
Data downloaded for ADIL


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Data downloaded for BA
Data downloaded for SLRX
Data downloaded for ALLR


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Data downloaded for WM
Data downloaded for CTNT


In [ ]:
data_dict['TSLA']

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-07-01,1.666667,1.728000,0.998667,1.329333,1.329333,968637000
2010-08-01,1.366667,1.478667,1.159333,1.298667,1.298667,225573000
2010-09-01,1.308000,1.544000,1.300000,1.360667,1.360667,270688500
2010-10-01,1.379333,1.458000,1.333333,1.456000,1.456000,98217000
2010-11-01,1.462667,2.400000,1.403333,2.355333,2.355333,424726500
...,...,...,...,...,...,...
2024-03-01,200.520004,204.520004,160.509995,175.789993,175.789993,1898904200
2024-04-01,176.169998,198.869995,138.800003,183.279999,183.279999,2478540800
2024-05-01,182.000000,187.559998,167.750000,178.080002,178.080002,1668180900
